# 加速：非同步爬蟲

* 了解非同步爬蟲加速原理與實作

## 作業目標

* 比較一下非同步爬蟲跟多線程爬蟲的差異是什麼？各自的優缺點為何？

1. 非同步：程式執行當下（不用等到執行完畢）就繼續往下執行
2. 多線程：程式會分成兩個子程式平行執行

### Python 中的非同步爬蟲

In [ ]:
import aiohttp
import asyncio
import nest_asyncio
nest_asyncio.apply()

async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text()

async def main():
    async with aiohttp.ClientSession() as session:
        html = await fetch(session, 'http://python.org')
        print(html)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())
loop.close()

### 多線程爬蟲

In [1]:
import requests, time

URL = 'https://morvanzhou.github.io/'

def normal():
    for i in range(2):
        r = requests.get(URL)
        url = r.url
        print(url)

t1 = time.time()
normal()
print("Normal total time:", time.time()-t1)


https://morvanzhou.github.io/
https://morvanzhou.github.io/
Normal total time: 0.420212984085083


In [2]:
import aiohttp, asyncio
import nest_asyncio
nest_asyncio.apply()


async def job(session):
    response = await session.get(URL)                               #等待並切換
    return str(response.url)

async def main(loop):
    async with aiohttp.ClientSession() as session:                  #官網推薦建立Session的形式,也可以直接用request
        tasks = [loop.create_task(job(session)) for _ in range(2)]
        finished, unfinished = await asyncio.wait(tasks)            #收集完成的結果,會返回完成的和沒完成的,等全部都完成了才返回
        all_results = [r.result() for r in finished]                #獲取所有結果
        print(all_results)

t1 = time.time()
loop = asyncio.get_event_loop()
loop.run_until_complete(main(loop))
# loop.close()
print("Async total time:", time.time() - t1)

['https://morvanzhou.github.io/', 'https://morvanzhou.github.io/']
Async total time: 0.08722615242004395
